In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from keras import losses, optimizers, metrics, layers, models, activations
from keras import backend as K
from keras.preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
import glob
from pathlib import Path
from sklearn.model_selection import train_test_split
import tensorflow as tf
import os
from keras import applications

In [18]:
path = "C:\\Users\\USER\\Documents\\Inteligentes\\dataexport.xlsx"
data_ori = pd.read_excel(path)
correl =data_ori.corr()


In [39]:
imgPath="C:\\Users\\USER\\Documents\\Inteligentes\\images_muestra"
fpaths = list(Path(imgPath).rglob('*.jpg'))
fpaths_s = pd.Series(fpaths,name='fpaths')
fpaths_s=fpaths_s.sort_values()
fpaths_s.replace("\t"," ", regex=True)
fpaths_s=fpaths_s.apply(lambda x: str(x));
for x in fpaths_s:
    if type(x) is not str:
        str(x)   
        
ds_selected=data_ori.filter(['frame','filter','brightness','image_quality'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(fpaths_s, ds_selected, test_size=0.15)
data_train = pd.concat([X_train, y_train ], axis=1);
data_test = pd.concat([X_test, y_test ], axis=1);

data_test['fpaths']=data_test['fpaths'].astype('|S')
print(data_train.shape)
print(data_test.shape)
data_train['fpaths']=data_train['fpaths'].apply(lambda x: str(x))

def get_label(file_path):
    parts=str(file_path,'utf-8')
    #print(parts)
    return parts

data_test['fpaths']=data_test['fpaths'].apply(lambda x: get_label(x))

(1064, 5)
(188, 5)


In [40]:
model_cnn = applications.MobileNet(include_top=False, weights='imagenet' , input_shape=(256, 256, 3))

In [41]:
batch_size=1

train_datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2)
test_datagen = ImageDataGenerator(zoom_range=0.2)



In [42]:
train_generator =train_datagen.flow_from_dataframe(
    dataframe=data_train, 
    directory=imgPath,
    x_col="fpaths",
    y_col=['frame','filter','brightness','image_quality'], 
    class_mode="other",
    batch_size=batch_size
)

Found 1064 validated image filenames.


In [43]:
test_generator =test_datagen.flow_from_dataframe(
    dataframe=data_test, 
    directory=imgPath,
    x_col="fpaths",
    y_col=['frame','filter','brightness','image_quality'], 
    class_mode="other",
    
)

Found 188 validated image filenames.


In [54]:
bottleneck_features_train = model_cnn.predict_generator( train_generator)

In [55]:
bottleneck_features_test = model_cnn.predict_generator(test_generator)

In [46]:
bottleneck_features_train.shape
bottleneck_features_test.shape

(188, 8, 8, 1024)

In [47]:
np.save('bottleneck_features_train.npy',
        bottleneck_features_train)
np.save('bottleneck_features_test.npy',
        bottleneck_features_test)

In [48]:
train_data = np.load('bottleneck_features_train.npy')
validation_data = np.load('bottleneck_features_test.npy')
print(bottleneck_features_train.shape)
print(bottleneck_features_test.shape)

(1064, 8, 8, 1024)
(188, 8, 8, 1024)


In [49]:
train_labels = data_train.copy()
validation_labels = data_test.copy()


In [75]:
model_relu = Sequential()
model_relu.add(Flatten(input_shape=train_data.shape[1:]))
model_relu.add(Dense(256, activation='relu'))
model_relu.add(Dense(256, activation='relu'))
model_relu.add(Dropout(0.1))
model_relu.add(Dense(128, activation='relu'))
model_relu.add(Dense(128, activation='relu'))
model_relu.add(Dropout(0.2))
model_relu.add(Dense(64, activation='relu'))
model_relu.add(Dense(64, activation='relu'))
model_relu.add(Dense(32, activation='relu'))
model_relu.add(Dense(32, activation='relu'))
model_relu.add(Dropout(0.2))
model_relu.add(Dense(16, activation='relu'))
model_relu.add(Dense(16, activation='relu'))
model_relu.add(Dense(8, activation='relu'))
model_relu.add(Dropout(0.2))
model_relu.add(Dense(4, activation='sigmoid'))
model_relu.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_16 (Flatten)         (None, 65536)             0         
_________________________________________________________________
dense_181 (Dense)            (None, 256)               16777472  
_________________________________________________________________
dense_182 (Dense)            (None, 256)               65792     
_________________________________________________________________
dropout_53 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_183 (Dense)            (None, 128)               32896     
_________________________________________________________________
dense_184 (Dense)            (None, 128)               16512     
_________________________________________________________________
dropout_54 (Dropout)         (None, 128)             

In [51]:
train_labels=train_labels.drop('fpaths',axis=1)
validation_labels=validation_labels.drop('fpaths',axis=1)


In [74]:
optimizer = optimizers.RMSprop(learning_rate=0.0001, rho=0.9)
model_relu.compile(optimizer=optimizer,
              loss='binary_crossentropy', metrics=['accuracy'])

model_relu.fit(train_data, train_labels,
          epochs=4,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))

Train on 1064 samples, validate on 188 samples
Epoch 1/4
1064/1064 [==============================] - 346s 325ms/step - loss: 0.7051 - accuracy: 0.5186 - val_loss: 0.6688 - val_accuracy: 0.5944
Epoch 2/4
1064/1064 [==============================] - 349s 328ms/step - loss: 0.6529 - accuracy: 0.5670 - val_loss: 0.6147 - val_accuracy: 0.5824
Epoch 3/4
1064/1064 [==============================] - 348s 327ms/step - loss: 0.6022 - accuracy: 0.5860 - val_loss: 0.5716 - val_accuracy: 0.6157
Epoch 4/4
1064/1064 [==============================] - 347s 326ms/step - loss: 0.5244 - accuracy: 0.5881 - val_loss: 0.4780 - val_accuracy: 0.6157


In [ ]:
for x_batch, y_batch in test_generator:
    print(y_batch)
    print(model_relu.predict_proba(model_cnn.predict(x_batch)))
